In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="ChatBot_with_chain_Prompt_template"

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

# Build the Prompt

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

First, let's add in a system message. To do this, we will create a ChatPromptTemplate. We will utilize MessagesPlaceholder to pass all the messages in.


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [4]:
# Build the chain
chain = prompt | model

Note that this slightly changes the input type - rather than pass in a list of messages, we are now passing in a dictionary with a messages key where that contains a list of messages.

In [5]:
from langchain_core.messages import HumanMessage

response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"})

In [6]:
response

AIMessage(content='¡Hola, Bob! ¿Cómo puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 33, 'total_tokens': 44}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2d7c831-2236-47f9-b925-545baf73bb24-0')

# Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. 


# Define message store


In [7]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# Function to get session history
This function is expected to take in a session_id and return a Message History object. This session_id is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain


In [8]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [9]:
# Function to generate a unique session ID
import uuid

def generate_session_id() -> str:
    return str(uuid.uuid4())

In [10]:
# Generating a dynamic session ID
session_id_1 = generate_session_id()

# Define Config

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a session_id. 

In [11]:
config_1 = {"configurable": {"session_id": session_id_1}}

In [12]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config_1,
)

In [13]:
response

AIMessage(content='¡Hola, Todd! ¿Cómo puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 33, 'total_tokens': 44}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed2fe52e-c5bd-42b0-9991-1f61f9829544-0')

In [14]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Whats my name?")], "language": "Hindi"},
    config=config_1,
)

In [15]:
response

AIMessage(content='आपका नाम टॉड है।', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 56, 'total_tokens': 64}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-95a0eaeb-db1c-4a3c-81f1-d8929800ed16-0')

In [16]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What was my last question?")], "language": "English"},
    config=config_1,
)

In [17]:
response

AIMessage(content='Your last question was "What\'s my name?"', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 78, 'total_tokens': 87}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe345dd4-0fd3-4b2f-9d57-2df42e711993-0')

In [18]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 2 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='Your last two questions were:\n\n1. "What was my last question?"\n2. "What’s my name?"', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 103, 'total_tokens': 126}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'stop', 'logprobs': None}, id='run-266b1d81-6c86-41f0-b669-c37d7003516e-0')

In [19]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 2 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='Your last two questions were:\n\n1. "What was my last question?"\n2. "Tell me my last 2 questions?"', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 142, 'total_tokens': 168}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-93d4fc7c-be27-4fe4-b539-622982018c79-0')

In [20]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Tell me my last 70 questions?")], "language": "English"},
    config=config_1,
)
response

AIMessage(content='I’m sorry, but I can only provide a limited history of our conversation, which includes the most recent questions you’ve asked. So far, you\'ve asked:\n\n1. "hi! I\'m todd"\n2. "What’s my name?"\n3. "What was my last question?"\n4. "Tell me my last 2 questions?"\n5. "Tell me my last 2 questions?" (repeated)\n\nIf you have more questions or need information on a specific topic, feel free to ask!', response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 184, 'total_tokens': 285}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_db4a9208a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-94778920-cc75-4f0e-ac94-c9b45e62684a-0')

In [21]:
# Generating a dynamic session ID
session_id_2 = generate_session_id()
config_2 = {"configurable": {"session_id": session_id_2}}

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Whats my name?")], "language": "Hindi"},
    config=config_2,
)
response

AIMessage(content='मुझे आपके नाम के बारे में जानकारी नहीं है। क्या आप मुझे अपना नाम बता सकते हैं?', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 33, 'total_tokens': 53}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-94dc4a02-5913-49e7-966d-a3244f4051e1-0')

# Important! The messages are being stored in the "store"


You can see the content here.



In [23]:
store

{'1fd27655-017d-4fea-9970-762ce7175b43': InMemoryChatMessageHistory(messages=[HumanMessage(content="hi! I'm todd"), AIMessage(content='¡Hola, Todd! ¿Cómo puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 33, 'total_tokens': 44}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed2fe52e-c5bd-42b0-9991-1f61f9829544-0'), HumanMessage(content='Whats my name?'), AIMessage(content='आपका नाम टॉड है।', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 56, 'total_tokens': 64}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-95a0eaeb-db1c-4a3c-81f1-d8929800ed16-0'), HumanMessage(content='What was my last question?'), AIMessage(content='Your last question was "What\'s my name?"', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 78, 'total_tokens': 87}, 'mo